In [38]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import numpy as np


mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0


model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(filters=48, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3158 - accuracy: 0.9039 - val_loss: 0.1122 - val_accuracy: 0.9693
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1109 - accuracy: 0.9664 - val_loss: 0.0822 - val_accuracy: 0.9770
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0821 - accuracy: 0.9750 - val_loss: 0.0720 - val_accuracy: 0.9797
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0676 - accuracy: 0.9792 - val_loss: 0.0792 - val_accuracy: 0.9780


In [47]:
import tensorflow as tf
import numpy as np

def prune_weights(model, pruning_threshold=0.1):
    pruned_model = tf.keras.models.clone_model(model)
    pruned_model.set_weights(model.get_weights())  

    for layer, pruned_layer in zip(model.layers, pruned_model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            weights = layer.get_weights()
            kernel_weights, bias = weights[0], weights[1]
            
            # IMPORTANCE SCORE USING L1 NORM ON INDIVIDUAL WEIGHTS
            importance_scores = np.abs(kernel_weights)
            threshold = np.percentile(importance_scores, pruning_threshold * 100)
            
            # IDENTIFY THE WEIGHTS TO BE PRUNED
            weights_to_prune = importance_scores < threshold
            
            # CHECK THE WEIGHTS GONNA BE PRUNED
            print("\n weights to be pruned:", weights_to_prune.sum(), len(weights_to_prune))
            
            # PRUNE THE WEIGHTS BY SETTING THEM TO ZERO
            kernel_weights[weights_to_prune] = 0.0
            
            # UPDATE THE WEIGHT
            pruned_layer.set_weights([kernel_weights, bias])

    return pruned_model


In [51]:
pruned_model = prune_filters(model, pruning_threshold=0.1)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
pruned_model.fit(train_images, train_labels, epochs=5, validation_split=0.2)


 filters to be pruned: [False False False False False False False  True False False  True False] 12

 filters to be pruned: [False False  True False False False False False False False False False
 False False False  True False False False False False  True False False] 24

 filters to be pruned: [False False False False False  True False False False False False False
 False False  True False False False False False False  True False False
 False False False  True False False False False False False  True False
 False False False False False False False False False False False False] 48
Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 1.3697 - accuracy: 0.8916 - val_loss: 1.7105 - val_accuracy: 0.8404
Epoch 2/5
1500/1500 [==============================] - 5s 4ms/step - loss: 1.9637 - accuracy: 0.8086 - val_loss: 2.0370 - val_accuracy: 0.8394
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 1.9836 - accuracy: 0.8200 - val_loss: 2.0040

In [52]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0739 - accuracy: 0.9751


[0.0739019513130188, 0.9750999808311462]

In [53]:
pruned_model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 2.3007 - accuracy: 0.5505


[2.300748825073242, 0.5504999756813049]